# In this notebook we'll have a look at time-series and reccurent neural nets (RNN) and in particular LSTM

useful links:
- http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- https://towardsdatascience.com/recurrent-neural-networks-d4642c9bc7ce
- https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

We'll first test how a simple LSTM network can learn to predict y[n + 1] based on a synthetical a cosine wave with exponential amplitude. We'll then have a look at a time-series from invasive pressure measurements.

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math
%matplotlib inline

t_start = 0
t_end = 2*np.pi
N = 1001
t = np.linspace(t_start, t_end, N)
y = np.cos(5*t)*np.exp(2*t/t_end)

case = "exponential"

if case == "P":
    data = np.genfromtxt("data/run_5")
    t = data[:, 0]
    y = data[:, 1]
plt.figure()
plt.plot(t, y)

y = y[:, np.newaxis] # turn 1D array into 2D array of shape (N, 1)


print(np.shape(data), np.shape(y))


In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    # dataX contain y_value of current time-step in addition to the previous (look_back - 1) time-steps
    # dataY contain the y_value of the next time-step
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 10
epochs = 10
trainFrac = 0.5
LSTM_blocks = 4

scaler = MinMaxScaler(feature_range=(0, 1))
y = scaler.fit_transform(y)
# split into train and test sets
train_size = int(len(y) * trainFrac)
test_size = len(y) - train_size
y_train, y_test = y[0:train_size,:], y[train_size:len(y),:]


In [ ]:
trainX, trainY = create_dataset(y_train, look_back)
testX, testY = create_dataset(y_test, look_back)
# turn 2D (N_train, look_back) array into 3D array of shape (N_train, 1, look_back)
# dims should be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1])) 
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
trainY = trainY[:, np.newaxis] # turn 1D array into 2D array of shape (N, 1)
testY = testY[:, np.newaxis] # turn 1D array into 2D array of shape (N, 1)

In [ ]:
model = Sequential()
model.add(LSTM(LSTM_blocks, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=epochs, batch_size=1, verbose=2)
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


In [ ]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[:,0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[:,0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
# shift train predictions for plotting
trainPredictPlot = np.empty_like(y)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(y)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(y)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(y))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()